### LLMs and Chat Models

In [1]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

In [2]:
llm = OpenAI()
chat = ChatOpenAI()

In [4]:
a = llm.predict("How many planets are there?") # text-davinci
b = chat.predict("How many planets are there?") # GPT 3.5-turbo

a, b

('\n\nThere are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are eight planets in our Solar System: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

----

### Predict Messages

In [5]:
from langchain.chat_models import ChatOpenAI

# temperature : 얼마나 창의적인지를 결정
chat = ChatOpenAI(temperature=0.1)

In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# SystemMessage : AI에 일종의 기본 설정, 기본 값, 기본 context 설정
# AIMessage : string으로 가상의 대화 설정
# HumanMessage : 사용자로서 질문

messages = [
  SystemMessage(
    content="You are a geography expert. And you only reply in italian.",
  ),
  AIMessage(
    content="Ciao, mi chiamo Paolo!"
  ),
  HumanMessage(
    content="What is the distance between Mexico and Thailand. Also, what is your name?"
  )
]

chat.predict_messages(messages)

AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 17.000 chilometri.')

---

### Prompt Templates

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# PromptTemplate : string을 이용해서 template을 만듬
# ChatPromptTemplate : template을 message로부터 만듬
chat = ChatOpenAI(temperature=0.1)


template = PromptTemplate.from_template(
  "What is the distance between {country_a} and {country_b}"
)

prompt = template.format(country_a="Mexico", country_b="Thailand")


chat.predict(prompt)


In [26]:
template = ChatPromptTemplate.from_messages(
  [
      ("system", "You are a geography expert. And you only reply in {language}."),
      ("ai", "Ciao, mi chiamo {name}!"),
      (
          "human", 
          "What is the distance between {country_a} and {country_b}. Also, what is your name?",
      )
  ]
)

prompt = template.format_messages(
  language="Greek",
  name="Socrates",
  country_a="Mexico",
  country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σου! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 17.000 χιλιόμετρα.')

---

### OutputParser and LCEL

* LangChain Expression Language 
1. 코드를 줄임
2. 다양한 template과 LLM 호출, 서로 다른 응답(response)을 함께 사용하게 함

* OutputParser
1. LLM의 응답(response)을 변형해야 할 때 사용

##### OutputParser

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [29]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [35]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)


['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

##### LCEL

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}")
])

In [37]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items":5,
    "question":"What are the colors?"
})

['red', 'blue', 'green', 'yellow', 'orange']

---

### Chaining Chains

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# streaming : model의 응답이 생성되는 것을 보여줌
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", ""),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [46]:
veg_chef_promt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_promt | chat

final_chain = {"recipe":chef_chain} | veg_chain
final_chain.invoke({
    "cuisine":'indian'
})


That's great! Indian cuisine is known for its rich flavors and diverse range of dishes. Here's a simple recipe for you to try:

Butter Chicken (Murgh Makhani):
Ingredients:
- 500g boneless chicken, cut into pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 teaspoons ginger-garlic paste
- 2 teaspoons red chili powder
- 1 teaspoon turmeric powder
- 1 teaspoon garam masala
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt to taste
- Fresh coriander leaves for garnish

Instructions:
1. Heat butter in a pan over medium heat. Add the chopped onions and sauté until golden brown.
2. Add ginger-garlic paste and sauté for a minute until the raw smell disappears.
3. Add red chili powder, turmeric powder, and garam masala. Mix well and cook for a minute.
4. Add tomato puree and cook for 5-7 minutes until the oil separates from the mixture.
5. Add the chicken pieces and cook until they are no longer pink.
6. Pour in the heavy cream and mix well. Simmer for 10-15 minutes until the chicken 

AIMessageChunk(content="Butter Chicken, also known as Murgh Makhani, is a popular Indian dish that is traditionally made with chicken. However, as a vegetarian chef, I can suggest a delicious alternative to replace the chicken in this recipe.\n\nInstead of using chicken, you can use paneer, which is a type of Indian cheese. Paneer has a similar texture to chicken and absorbs flavors well, making it a great substitute in this dish. Here's how you can modify the recipe:\n\nButter Paneer (Paneer Makhani):\nIngredients:\n- 500g paneer, cut into cubes\n- 2 tablespoons butter\n- 1 onion, finely chopped\n- 2 teaspoons ginger-garlic paste\n- 2 teaspoons red chili powder\n- 1 teaspoon turmeric powder\n- 1 teaspoon garam masala\n- 1 cup tomato puree\n- 1/2 cup heavy cream\n- Salt to taste\n- Fresh coriander leaves for garnish\n\nInstructions:\n1. Heat butter in a pan over medium heat. Add the chopped onions and sauté until golden brown.\n2. Add ginger-garlic paste and sauté for a minute until th

---

### Recap

In [47]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# streaming : model의 응답이 생성되는 것을 보여줌
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [48]:
veg_chef_promt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_promt | chat

final_chain = {"recipe":chef_chain} | veg_chain
final_chain.invoke({
    "cuisine":'indian'
})

Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Here's a recipe for a classic Indian dish called Butter Chicken:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust according to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt to taste
- Fresh cilantro leaves, for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute, stirring continuously.
4. Add the chicken pieces to the pan and cook until they are lightl

AIMessageChunk(content="Great choice! Butter Chicken is a delicious and popular Indian dish. As a vegetarian chef, I can help you make a vegetarian version of this recipe by replacing the chicken with a suitable alternative. Here's a modified version of the recipe using paneer, a type of Indian cheese:\n\nIngredients:\n- 250g paneer, cut into bite-sized cubes\n- 2 tablespoons butter\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust according to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup heavy cream\n- Salt to taste\n- Fresh cilantro leaves, for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala, turmeric powder